In [58]:
import requests
import time
import logging
import os
import jwt
import json
import datetime
import uuid

def get_device_code():
    bodyDeviceCode = {
        "client_id": "1950a258-227b-4e31-a9cf-717495945fc2",
        "scope": "https://management.core.windows.net//.default offline_access profile openid",
        "claims": '{"access_token":{"xms_cc":{"values":["CP1"]}}}'
    }
    deviceCodeResponse = requests.get("https://login.microsoftonline.com/organizations/oauth2/v2.0/devicecode", data=bodyDeviceCode).json()
    logging.info(deviceCodeResponse['message'])
    secondsUntilVerificationExpires = deviceCodeResponse['expires_in']
    secondsBetweenVerificationChecks = deviceCodeResponse['interval']
    device_code = deviceCodeResponse['device_code']
    return device_code, secondsBetweenVerificationChecks, secondsUntilVerificationExpires

def get_device_code_graph():
    bodyDeviceCode = {
        "client_id": "1950a258-227b-4e31-a9cf-717495945fc2",
        "scope": "https://graph.microsoft.com//.default offline_access openid profile",
        "claims": '{"access_token":{"xms_cc":{"values":["CP1"]}}}'
    }
    deviceCodeResponse = requests.get("https://login.microsoftonline.com/organizations/oauth2/v2.0/devicecode", data=bodyDeviceCode).json()
    logging.info(deviceCodeResponse['message'])
    secondsUntilVerificationExpires = deviceCodeResponse['expires_in']
    secondsBetweenVerificationChecks = deviceCodeResponse['interval']
    device_code = deviceCodeResponse['device_code']
    return device_code, secondsBetweenVerificationChecks, secondsUntilVerificationExpires

def get_access_token(device_code:str, secondsBetweenVerificationChecks:int, secondsUntilVerificationExpires:int)->str:
    bodyToken : dict[str, str]= {
        "client_id": "1950a258-227b-4e31-a9cf-717495945fc2",
        "scope": "https://management.core.windows.net//.default offline_access openid profile",        
        "claims": '{"access_token":{"xms_cc":{"values":["CP1"]}}}',
        "client_info": "1",
        "grant_type": "device_code",
        "device_code": device_code
    }
    while secondsUntilVerificationExpires > 0:
        time.sleep(secondsBetweenVerificationChecks)
        secondsUntilVerificationExpires -= secondsBetweenVerificationChecks
        token_response = requests.post("https://login.microsoftonline.com/organizations/oauth2/v2.0/token", data=bodyToken).json()
        if 'access_token' in token_response:
            return token_response['access_token']
        if 'error' in token_response:
            error_name = token_response['error']
            if error_name == "authorization_pending":
                continue
            logging.error(error_name)
            break
    return ""

def get_access_token_graph(device_code:str, secondsBetweenVerificationChecks:int, secondsUntilVerificationExpires:int)->str:
    bodyToken : dict[str, str]= {
        "client_id": "1950a258-227b-4e31-a9cf-717495945fc2",
        "scope": "https://graph.microsoft.com//.default offline_access openid profile",        
        "claims": '{"access_token":{"xms_cc":{"values":["CP1"]}}}',
        "client_info": "1",
        "grant_type": "device_code",
        "device_code": device_code
    }
    while secondsUntilVerificationExpires > 0:
        time.sleep(secondsBetweenVerificationChecks)
        secondsUntilVerificationExpires -= secondsBetweenVerificationChecks
        token_response = requests.post("https://login.microsoftonline.com/organizations/oauth2/v2.0/token", data=bodyToken).json()
        if 'access_token' in token_response:
            return token_response['access_token']
        if 'error' in token_response:
            error_name = token_response['error']
            if error_name == "authorization_pending":
                continue
            logging.error(error_name)
            break
    return ""

def create_secret(app_name:str, access_token:str):
    url = f"https://graph.microsoft.com/v1.0/applications/{app_name}/addPassword"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'passwordCredential': {
            'displayName': 'Client Secret',
            'endDateTime': (datetime.datetime.now() + datetime.timedelta(hours=1)).isoformat(),
            'keyId': str(uuid.uuid4())
        }
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload)).json()
    return response['id']

def store_secret_in_key_vault(vault_name:str, secret_id:str, access_token:str, subscription_id:str, resource_group:str):
    url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.KeyVault/vaults/{vault_name}/secrets/{secret_id}?api-version=2019-09-01"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    payload = {
        'value': secret_id,
        'tags': {
            'displayName': 'Client Secret'
        }
    }
    requests.put(url, headers=headers, data=json.dumps(payload))

def get_secret_from_key_vault(vault_name:str, secret_id:str, access_token:str, subscription_id:str, resource_group:str):
    url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.KeyVault/vaults/{vault_name}/secrets/{secret_id}?api-version=2019-09-01"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers).json()
    return response['value']


In [ ]:
logging.basicConfig(level=logging.INFO)
access_token = ""
if 'AZURE_ACCESS_TOKEN' in os.environ:
    access_token = os.environ['AZURE_ACCESS_TOKEN']
    # decode the jwt accessToken to ensure it is not expired
    decoded_token = jwt.decode(access_token, options={"verify_signature": False})
    # Check if the token will expire in the next 10 minutes
    if time.time() < decoded_token['exp'] - 100*60:
        print(access_token)
device_code, secondsBetweenVerificationChecks, secondsUntilVerificationExpires = get_device_code()
access_token:str = get_access_token(device_code, secondsBetweenVerificationChecks, secondsUntilVerificationExpires)
print(access_token)
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

In [ ]:
device_code_graph, secondsBetweenVerificationChecks, secondsUntilVerificationExpires = get_device_code_graph()
access_token_graph:str = get_access_token_graph(device_code_graph, secondsBetweenVerificationChecks, secondsUntilVerificationExpires)
print(access_token_graph)
headers_graph = {
    "Authorization": f"Bearer {access_token_graph}",
    "Content-Type": "application/json"
}

In [100]:
# get a list of all passwords for the app
app_id = "4746ae3b-8a2b-4dd3-836a-68772d6981fa" #object id
url = f"https://graph.microsoft.com/v1.0/applications/{app_id}"
response = requests.get(url, headers=headers_graph).json()
print(response)
passwordCredentials = response['passwordCredentials']


{'@odata.context': 'https://graph.microsoft.com/v1.0/$metadata#applications/$entity', 'id': '4746ae3b-8a2b-4dd3-836a-68772d6981fa', 'deletedDateTime': None, 'appId': 'ded7e350-97b8-481f-90c0-e7c9bd485964', 'applicationTemplateId': None, 'disabledByMicrosoftStatus': None, 'createdDateTime': '2024-02-22T17:51:53Z', 'displayName': 'onboard-arc-service', 'description': None, 'groupMembershipClaims': None, 'identifierUris': [], 'isDeviceOnlyAuthSupported': None, 'isFallbackPublicClient': None, 'nativeAuthenticationApisEnabled': None, 'notes': None, 'publisherDomain': 'slowjedi.com', 'serviceManagementReference': None, 'signInAudience': 'AzureADandPersonalMicrosoftAccount', 'tags': ['AzureArcSPN'], 'tokenEncryptionKeyId': None, 'uniqueName': None, 'samlMetadataUrl': None, 'defaultRedirectUri': None, 'certification': None, 'optionalClaims': None, 'servicePrincipalLockConfiguration': None, 'requestSignatureVerification': None, 'addIns': [], 'api': {'acceptMappedClaims': None, 'knownClientAppli

In [99]:

# loop through all the password keyIds and delete them
for password in passwordCredentials:
    key_id = password['keyId']
    print(key_id)
    url = f"https://graph.microsoft.com/v1.0/applications/{app_id}/removePassword"
    payload = {'keyId': key_id}
    response = requests.post(url, headers=headers_graph, data=json.dumps(payload)).json()
    print(response)



09a631fe-06d2-4152-94b5-bdfc69a5ef3e
{'error': {'code': 'Request_BadRequest', 'message': "No password credential found with keyId as '09a631fe-06d2-4152-94b5-bdfc69a5ef3e'", 'innerError': {'date': '2024-05-30T03:52:16', 'request-id': 'c8b58dc3-574a-45d0-b822-7dd56c8efbc2', 'client-request-id': 'c8b58dc3-574a-45d0-b822-7dd56c8efbc2'}}}


In [101]:
payload = {
    'passwordCredential': {
        'displayName': 'Client Secret',
        'endDateTime': (datetime.datetime.now() + datetime.timedelta(hours=1)).isoformat()
    }
}
password_response = requests.post(f"https://graph.microsoft.com/v1.0/applications/{app_id}/addPassword", headers=headers_graph, data=json.dumps(payload)).json()

In [102]:
secret_text = password_response['secretText']
secret_text

''

In [95]:
subscription_id = "d8b98a16-2943-4e64-91b9-01a182064f05" #input('Enter the subscription ID: ')
resourceGroupName = "default-RG" #input('Enter the resource group name: ')


In [96]:
machines_list_response = requests.get("https://api.securitycenter.windows.com/api/machines", headers=headers)

In [40]:
#import json
#print(json.dumps(response.json(), indent=4, sort_keys=True))
machines = machines_list_response.json()["value"]
test_vm_id = '70ee6084237c5f0d9c7cbd636d05ea5bbb610954'
# "powershell.exe -File .\onboard_arc.ps1 -ServicePrincipalClientSecret \"your_secret_here\""
cmdLine = 'powershell.exe -File .\\onboard_arc.ps1 -ServicePrincipalClientSecret "' + secret_text + '"'
body = {
    "Commands":[
        {
            "type":"GetFile",
            "params":[
                {
                "key":"Path",
                "value":cmdLine
                }
            ]
        },
    ],
    "Comment":"Testing Live Response API"
}

for machine in machines:    
    machine_id = machine["id"]
    if machine_id == "70ee6084237c5f0d9c7cbd636d05ea5bbb610954":
        computerDnsName = machine["computerDnsName"]
        print("Computer DNS Name: ", computerDnsName)
        arc_resource_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resourceGroupName}/providers/Microsoft.HybridCompute/machines/{computerDnsName}?api-version=2019-12-12"
        arc_resource_response = requests.get(arc_resource_url, headers=headers)
        status_code_if = arc_resource_response.status_code
        if status_code_if == 200:
            arc_resource_data = arc_resource_response.json()
            print(arc_resource_data)
            break
        #connection_status = arc_resource_data['properties']['status']
        #print("Connection status: ", connection_status)
        #run_live_response = requests.post(f"https://api.securitycenter.microsoft.com/API/machines/{machine_id}/runliveresponse", headers=headers, json=body)
        #live_response_id = run_live_response.json()["id"]


Computer DNS Name:  desktop-ih5bd0h


In [25]:
print(live_response_id)

#get the live response status
url = f"https://api.securitycenter.microsoft.com/api/machineactions/{live_response_id}"
response = requests.get(url, headers=headers)
print(response)
#status = response.json()["status"]

736a997d-64de-46a7-a745-3c0d6c98a19f
<Response [200]>


In [27]:
import time

#get the live response status
url = f"https://api.securitycenter.microsoft.com/api/machineactions/{live_response_id}"

while True:
    action_result_response = requests.get(url, headers=headers)
    if action_result_response.status_code in [200, 401]:
        print(action_result_response)
        break
    else:
        time.sleep(5)  # wait for 5 seconds before the next request

736a997d-64de-46a7-a745-3c0d6c98a19f
<Response [200]>


In [29]:
action_result = action_result_response.json()
print(action_result['computerDnsName'] + " " + action_result['status'])
#action_result

desktop-ih5bd0h Succeeded


{'@odata.context': 'https://api.securitycenter.microsoft.com/api/$metadata#MachineActions/$entity',
 'id': '736a997d-64de-46a7-a745-3c0d6c98a19f',
 'type': 'LiveResponse',
 'title': None,
 'requestor': 'yoda@slowjedi.com',
 'requestorComment': 'Testing Live Response API',
 'status': 'Succeeded',
 'machineId': '70ee6084237c5f0d9c7cbd636d05ea5bbb610954',
 'computerDnsName': 'desktop-ih5bd0h',
 'creationDateTimeUtc': '2024-05-29T22:00:33.5093044Z',
 'lastUpdateDateTimeUtc': '2024-05-29T22:02:21.96704Z',
 'cancellationRequestor': None,
 'cancellationComment': None,
 'cancellationDateTimeUtc': None,
 'errorHResult': 0,
 'scope': None,
 'externalId': None,
 'requestSource': 'Portal',
 'relatedFileInfo': None,
 'commands': [{'index': 0,
   'startTime': '2024-05-29T22:00:41.171256Z',
   'endTime': '2024-05-29T22:02:21.966403Z',
   'commandStatus': 'Completed',
   'errors': [],
   'command': {'type': 'GetFile',
    'params': [{'key': 'Path', 'value': 'C:\\Windows\\notepad.exe'}]}}],
 'troublesh